In [81]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [82]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [83]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity1.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance1.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
GDP = CSV.File(string(path,"GDP.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [84]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"]
penalty = 5; # ton/yr;

In [85]:
# scen_eff = CSV.File(string(path,"Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
# scen_eff = Matrix(scen_eff)[:, 2:end]
scen_eff = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/Scenarios/feasible_scen.csv", header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]

scen_d = CSV.File(string(path,"Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_d = Matrix(scen_d)[2:end]
scen_dmd = rand(scen_d, 19)


Omega = ones(Float64, ncty, 19, nproc)  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff
num_omega = size(Omega)[2];

In [86]:
scen_dmd = ones(19) * 0.001*164.98*(1.369*1e6)*2;

---

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [88]:
function subprob(x_hat, s) # s represents specific scenario 
    
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)
    
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]  # ncty * nproc constraints
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]  # ncty * ncth constraints
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]  # ncty * ncell constraints
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]  # nnoncell * nproc constraints
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]  # nbat * nproc constraints
    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]  # 1 constraint
    cstr_fmk_o = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in 1:ncty-1];  # ncty-1 constraint (accidently the final market (U.S.) locates at the last position in the vector which is 30th)
    
    
    
    ####################### 
    tranD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
#             arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
#         tranD[j] = arc_emi - arc_seq
        tranD[j] = arc_emi
    end
    
    
    
    ####################### 
    @objective(model, Min, sum(tranD)+unmeet*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    
    ####################### 
    
    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]
    fmko_alp = [getdual(con) for con in cstr_fmk_o]

    Moppi = reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    Mcthpi = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode] * (-1)
    Mcellpi = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell] * (-1)
    Mncellpi = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell] * (-1)
    Mbty = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery] * (-1)
    Mtx_PI = hcat(Mcthpi, Mcellpi, Mncellpi, Mbty, zeros(ncty)) + Moppi
    Mtx_ALP = hcat(zeros(ncty, nproc-1), vcat(fmko_alp, fmk_alp*(scen_dmd[s]-sub_unmeet)));

    
    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>Mtx_PI, "alp"=>Mtx_ALP]);
    return result
    
end

subprob (generic function with 1 method)

In [89]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.6*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
#     if cuts != []
#         for cut in cuts
#             G = cut["gradient"]
#             g = cut["intersection"]
#             @constraint(model, sum(x .* G) + theta >= g)
#         end
#     end
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G * ones(nproc,1)) + theta >= sum(g * ones(nproc,1))); 
        end
    end
    
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

In [90]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    for s in 1:num_omega
        res = subprob(x, s)        
        gradient = -1 * res["pi"]
        push!(G, gradient)
        push!(g, res["alp"])
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    z_hat = sum(proD - proS) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat
end

add_cuts (generic function with 1 method)

In [91]:
function main(toler)
    i = 0
    z_ub = 1e10
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

-----

In [92]:
M = -1e6
toler = 0.01

0.01

In [93]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-1.1606048071532747e7

In [94]:
toler * min(abs(z_ub), abs(z_lb))

116060.48071532747

In [95]:
while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
    new_cut, z_hat = add_cuts(x_hat)
    if z_hat < z_ub
        z_ub = z_hat
        x_opt = x_hat
    end

    push!(cuts, new_cut)
    println(z_ub - z_lb)
    

    res1 = masterprob(cuts)
    z_lb = res1["z_lb"]
    x_hat = res1["x_hat"]

    i += 1
end

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

1.4534490201541036e6
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-

1.4534490201541036e6
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-

1.4534490201541036e6
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-

LoadError: InterruptException:

In [100]:
3 ∉ [3]

false

In [97]:
res = [cuts[1]]
for c in cuts
    if c ∉ res
        push!(res, c)

true

---

In [35]:
M = -1e10

-1.0e10

In [36]:
i = 0
z_ub = 1e10
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-1.0010606048071533e10

In [37]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

(Dict("gradient" => [0.36500392260271103 0.6103064868659765 … -0.0 1.257841094499076; -0.0 -0.0 … -0.0 1.06511273022561; … ; 0.30834246682975897 0.39056712465102805 … -0.0 1.1884497169575134; 1.939233608239837 3.254286041904471 … 1.4529689566817123 1.6527037642759974], "intersection" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.1613624939666928e6]), -7.881786070746445e6)

In [38]:
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat
end

push!(cuts, new_cut);

In [39]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= 0.6*capacity[!, 2:end][i,k])
    end
end

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];


###########
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1);

Academic license - for non-commercial use only - expires 2024-12-26


In [40]:
G = 0
g = 0
for cut in cuts
    G = cut["gradient"]
    g = cut["intersection"]
end
@constraint(model, sum(x .* G * ones(nproc,1)) + theta >= sum(g * ones(nproc,1)));

In [41]:
@objective(model, Min, sum(proD-proS)+theta);

In [42]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 465 rows, 451 columns and 1545 nonzeros
Model fingerprint: 0x0bb8ee3e
Coefficient statistics:
  Matrix range     [1e-04, 3e+00]
  Objective range  [7e-03, 3e+02]
  Bounds range     [1e+10, 1e+10]
  RHS range        [1e+02, 6e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 457 rows and 373 columns
Presolve time: 0.00s
Presolved: 8 rows, 78 columns, 137 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.6281445e+07   1.358389e+06   0.000000e+00      0s
       8   -1.1111425e+07   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds
Optimal objective -1.111142486e+07

User-callback calls 53, time in user-callback 0.00 sec


In [43]:
x_hat

30×15 Matrix{Float64}:
  3720.0      0.0  0.0            0.0  …       0.0             0.0
 36600.0   4200.0  0.0            0.0          0.0             0.0
  1320.0      0.0  1.05786e5  40200.0          0.0             0.0
   300.0   1860.0  0.0        68220.1          0.0             0.0
 23400.0      0.0  0.0            0.0     600000.0             0.0
 11376.7   1320.0  0.0            0.0  …       0.0             0.0
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0  87000.0  0.0            0.0          0.0             0.0
     0.0    650.4  0.0            0.0          0.0             0.0
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0      0.0  0.0            0.0  …       0.0             1.05867e5
     0.0      0.0  0.0            0.0          0.0             0.0
     0.0   5672.4  0.0            0.0          0.0             0.0
     ⋮                                 ⋱                  
     0.0      0.0  0.0            0.0    

In [44]:
x_hat = JuMP.value.(x)
theta_hat = JuMP.value.(theta)
z_lb = JuMP.objective_value(model)

result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])

Dict{String, Any} with 2 entries:
  "x_hat" => [3720.0 0.0 … 0.0 0.0; 36600.0 4200.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0…
  "z_lb"  => -1.11114e7